In [1]:

import tifffile
from pyometiff import OMETIFFReader
import squidpy as sq
from scipy import ndimage
import skimage.io
import skimage.measure
import skimage.segmentation
import numpy as np
import argparse
import os
import torch 
from cellpose import models,core
import logging
#models_logger = logging.getLogger(__name__)
#logging.basicConfig(level=logging.DEBUG)
#logging.getLogger('cellpose').setLevel(logging.DEBUG)
from cellpose import io 
logger = io.logger_setup()
from imageio.v2 import imread


2023-11-08 08:42:35,894 [INFO] WRITING LOG OUTPUT TO /home/gargerd/.cellpose/run.log
2023-11-08 08:42:35,895 [INFO] 
cellpose version: 	2.2.3.dev84+g167f273 
platform:       	linux 
python version: 	3.10.0 
torch version:  	1.13.1


In [2]:
fn='../xenium_data/20230808__140639__2311-01_Sachs_Panel1/output-XETG00050__0003370__P3_D__20230808__140759/morphology_mip.ome.tif'
#reader=OMETIFFReader(fpath=fn)
#img,metadata,xml_metadata=reader.read()
img=imread(fn)
print('Input image shape:',img.shape)

use_GPU = core.use_gpu()
model = models.Cellpose(gpu=use_GPU,model_type='cyto')
hyperparams={"batch_size":1,"channel_axis":None,"z_axis":None,"invert":False,"normalize":True,
            "diameter":30.0,"do_3D":False,"anisotropy":None,"net_avg":False,"augment":False,"tile":True,
            "tile_overlap":0.1,"resample":True,"interp":True,"flow_threshold":0.0,"cellprob_threshold":0.0,
            "min_size":15,"stitch_threshold":0.0,"rescale":False,"progress":False,"model_loaded":False}

res, _, _, _ = model.eval(
            img,
            channels=[0, 0],
            **hyperparams)

Input image shape: (34056, 28436)
2023-11-08 08:42:49,848 [INFO] ** TORCH CUDA version installed and working. **
2023-11-08 08:42:49,849 [INFO] ** TORCH CUDA version installed and working. **
2023-11-08 08:42:49,849 [INFO] >>>> using GPU
2023-11-08 08:42:49,863 [INFO] >> cyto << model set to be used
2023-11-08 08:42:50,395 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)
2023-11-08 08:42:50,397 [INFO] ~~~ FINDING MASKS ~~~
2023-11-08 08:50:17,972 [INFO] >>>> TOTAL TIME 447.57 sec
